In [ ]:
!pip install transformers
from transformers import pipeline



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00


In [ ]:
generator = pipeline('text-generation', model = 'gpt2')
generator("I HATE MY JOB", max_length = 30, num_return_sequences=3)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I HATE MY JOB AND DO MY WORK! I WAS HATING MY JOB AND DO MY WORK!\n\nAnonymous 01/11/'},
 {'generated_text': 'I HATE MY JOB, I HATE MY JOB SADLY, I HATE MY JOB JING! IT WAS SO FUCK'},
 {'generated_text': "I HATE MY JOB OF NOT WORKING ON THIS GAME, BUT WHAT'S IT ALL ABOUT? HIGHLY READ THE REST HERE"}]

## Idea 1
Train text generator on Amazon bad reviews


In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: command not found: pip


Found cached dataset amazon_us_reviews (/Users/ameezingwongstein/.cache/huggingface/datasets/amazon_us_reviews/Digital_Video_Games_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  8.17it/s]


In [ ]:
import pandas as pd

data_df = pd.DataFrame.from_dict(dataset['train'])
data_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,Digital_Video_Games,2,2,3,0,0,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31
1,US,133437,R1WFOQ3N9BO65I,B00F4CEHNK,341969535,Xbox Live Gift Card,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome,2015-08-31
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,Digital_Video_Games,5,0,0,0,1,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31
3,US,113118,R3R14UATT3OUFU,B004RMK5QG,395682204,Playstation Plus Subscription,Digital_Video_Games,5,0,0,0,1,Five Stars,Perfect,2015-08-31
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome!,2015-08-31


In [ ]:
bad_reviews = data_df.loc[data_df["star_rating"] == 1]
len(bad_reviews)

24970

In [ ]:
bad_reviews["review_body"].head()

10    As has been written by so many others, I quick...
14                Lames purchase I almost never made...
24                                               Gross!
26           Not the best graphics :(<br />Dissapointed
30                                               Stolen
Name: review_body, dtype: object

In [ ]:
bad_reviews["message_len"]=[len(i) for i in bad_reviews["review_body"].values] # [0 for i in range(0,24970)]

/var/folders/y0/_r4pmpsx6qjbvc93xxxb0lbm0000gn/T/ipykernel_49208/1791466554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_reviews["message_len"]=[len(i) for i in bad_reviews["review_body"].values] # [0 for i in range(0,24970)]


In [ ]:
# dall=dall.mul(row.loc[0],axis=1)

In [ ]:
# [len(i) for i in bad_reviews["review_body"].values]

In [ ]:
bad_reviews_small=bad_reviews.loc[bad_reviews["message_len"]<128]

In [ ]:
bad_reviews_small.shape

(7671, 16)

In [ ]:
bad_reviews.shape

(24970, 16)

In [ ]:
# bad_reviews["message_len"]

In [ ]:
# bad_reviews["message_len"]

# Cleaning Data

In [ ]:
# a little sentiment analysis
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
sentiment_pipeline = pipeline("text-classification", model=model_name)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
sentiment_pipeline.__dict__

{'task': 'text-classification',
 'model': RobertaForSequenceClassification(
   (roberta): RobertaModel(
     (embeddings): RobertaEmbeddings(
       (word_embeddings): Embedding(64001, 768, padding_idx=1)
       (position_embeddings): Embedding(130, 768, padding_idx=1)
       (token_type_embeddings): Embedding(1, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): RobertaEncoder(
       (layer): ModuleList(
         (0): RobertaLayer(
           (attention): RobertaAttention(
             (self): RobertaSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): RobertaSelfOutput(
               (dense): Linear(in_fe

In [ ]:
review = bad_reviews_small["review_body"].values[0]
sentiment_pipeline(review)

[{'label': 'NEG', 'score': 0.9216219782829285}]

In [ ]:
cleaner_data = None

for index, bad_review in bad_reviews_small.iterrows():
    sentiment = sentiment_pipeline(bad_review["review_body"])[0]
    if (sentiment["label"] == "POSITIVE"):
        cleaner_data = bad_reviews.drop(index=index, axis = 0)
    elif sentiment["score"] < 0.8:
        cleaner_data = bad_reviews.drop(index=index, axis = 0)


In [ ]:
len(cleaner_data )

24969

In [ ]:
cleaner_data.to_csv("cleaner_data.csv")

# Retraining on top of transformers

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
import pandas as pd
cleaned_data = pd.read_csv("MyDrive/vide_noir/cleaner_data.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cleaned_data.review_body, test_size = 0.15)
# train.to_csv("train.csv")
# test.to_csv("test.csv")

In [ ]:
train.head()

22288    Black screen and sound after install. I have W...
2051     weird games - impossible to figure out - impos...
15826    The game was a lot of fun, but it started cras...
3293                                     WILL NOT DOWNLOAD
19830    I enjoy playing Bejeweled, however, I do not l...
Name: review_body, dtype: object

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset("MyDrive/vide_noir/train.csv",'MyDrive/vide_noir/test.csv',tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 29.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#
from transformers import Trainer, TrainingArguments
import torch



training_args = TrainingArguments(
    output_dir="MyDrive/vide_noir/logs", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.695200
1000,3.416800


TrainOutput(global_step=1404, training_loss=3.4908395968271457, metrics={'train_runtime': 1550.8228, 'train_samples_per_second': 28.955, 'train_steps_per_second': 0.905, 'total_flos': 2933264351232000.0, 'train_loss': 3.4908395968271457, 'epoch': 3.0})

In [ ]:
trainer.save_model("MyDrive/vide_noir/trained_model.pt")

# Playing with Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

OSError: ignored

In [ ]:
model = AutoModelForCausalLM.from_pretrained("MyDrive/vide_noir/trained_model")

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model = model, tokenizer = tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
generator("I HATE MY JOB", max_length = 150, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I HATE MY JOB OF NOT KNOW IF I WANT TO PLAY THIS OR A PATCH.  I STILL AM NOT THE SERVER THAT WON\'T BE ABLE TO PLAY THE GAME I BUY HERE IF IT WON\'T WORK LIKE WHAT MIGHT HELP YOU TO PLAY THIS GAME.\n2620,"The game is crap, I cannot get down to anything about how the missions worked and there was a ton of wasted effort put into the game.  The game has'},
 {'generated_text': 'I HATE MY JOB!"\n16962,Don\'t buy this game.  This game sucks and the graphics are terrible.  Don\'t buy it.  If you bought one of the other online games, you better be sure you know you aren\'t getting screwed and you will lose a lot of money.\n7577,"The games work fine for my PC. But I wouldn\'t recommend this game unless you really have high end gaming equipment. My daughter is an enthusiast with great'},
 {'generated_text': 'I HATE MY JOB."\n5137,"Didn\'t work. Tried everything to find out that I had to have a steam account to use the game."\n23754,"I am a huge EA fan. As a big fan of SimCit

In [ ]:
generator("Nobody understands me", max_length = 150, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Nobody understands me. I feel a lot insulted. I\'m a software developer and I love a game that is free to play. But I\'ve really looked forward to playing SimCity. This is what my brain told me about that game. explaining myself.\n20590,When the game first started, it was \\\\""a mess\\\\"". It was hard to download what you paid and it took way too long. It was a waste of money and I\'m trying to get a refund for it\n18062,It wasn\'t worth the money. I wouldn\'t recommend this. Just don\'t buy this. Even if you buy it you will have it for less.\n2592,"I\'m trying to play this game because'},
 {'generated_text': 'Nobody understands me.<br />Do not waste your time here. Save your money, buy something else."\n6177,"I\'ve played a total of about 80% of this game, but I haven\'t been able to finish it.  If this is a game for older players, I don\'t see how anyone would enjoy it.  Also, because the graphics for this game are awful, I couldn\'t get a full review on it."\n

# Comparison with untuned GPT-2

In [ ]:
generator_original = pipeline('text-generation', model = 'gpt2')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I HATE MY JOB.I HATE MY JOB.I HATE MY JOB.I HATE MY JptiveITY.I'},
 {'generated_text': 'I HATE MY JOB!!!\n\nI HATE MY JOB!!!\n\nI HATE MY JOB!!!\n\nI HATE'},
 {'generated_text': 'I HATE MY JOB."\n\nMr. Trump, like his brother, has struggled with his acceptance speech amid an avalanche of questions over his'}]

In [ ]:
generator_original("I hate my job", max_length = 50, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I hate my job to make those who are sick and injured feel comfortable or comfortable if their employer isn't around. This is the situation for me myself. What I do, I don't take responsibility for it, and I hope everyone understands that."},
 {'generated_text': 'I hate my job. Fuck all of this. You did this.\n\nMELISSA: What about when you went back home? What did you see a lot of during your stay in jail?\n\nSHAKOTTI: We'},
 {'generated_text': 'I hate my job," she told me.\n\nShe said she\'s glad she has quit working as a nurse for 20 years, but she also doesn\'t care who wins this fight to take women\'s health care. "But I hate the women'}]

In [ ]:
generator("I hate my job", max_length = 50, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I hate my job and I just want a refund.\n18001,Game doesn't play and you will never get a refund either!!!! I have so many other issues from this download version that you will not be able to play.\n15996"},
 {'generated_text': 'I hate my job and I hate the game - not that I hate it, I really do and still do - and I have nothing to do but like this."\n13761,Would not install.  Didn\'t know it required an x'},
 {'generated_text': 'I hate my job, I have more money, but I did not get this code to show my email, i just purchased the physical disc and will not play it."\n2613,"Does not work with XP... I deleted it a few months'}]